In [2]:
import pandas as pd
import nltk

### Tokenize and simplify the ingredients

In [2]:
# Upload trained dataset
train_ingr=pd.read_csv('data/shortened_train.csv')

train_ingr['full form']=pd.Series([x.lstrip() for x in train_ingr['full form']])
train_ingr['short form']=pd.Series([x.lstrip() for x in train_ingr['short form']])
train_ingr.index=train_ingr['full form']

groups = train_ingr.groupby(level=train_ingr.index.names) 
train_ingr=groups.last()


In [285]:
# read the full list of 3700 ingredients
ingredients=pd.read_csv('data/ingredients.csv')
ingredients.drop('Column1', axis=1, inplace=True)

In [287]:
ingredients.head()

,full name,stemmed (used for the dataset),substitute 1,substitute 2
0,cremini mushrooms,crimini mushrooms,common mushroom,NaN
1,caramel,caramel,NaN,NaN
2,maldon sea salt,sea salt,salt,NaN
3,orecchiette,orecchiette,NaN,NaN
4,baby bok choy,bok choy,NaN,NaN


In [ ]:
# create the dataset of ingredients

ingredients['split']=pd.Series([x.split() for x in new_ingredients])

ingredients['pos']=pd.Series([[xx[1] for xx in nltk.pos_tag(x)] for x in ingredients['split']])

ingredients['train']=pd.Series([int(x in list(train_ingr['full form'])) for x in ingredients['full name']])

ingredients['short']=pd.Series([train_ingr.loc[x]['short form'] 
                                if (x in train_ingr.index)&(x in new_ingredients) else 'na' for x in ingredients['full name'] ])

ingredients['short_split']=pd.Series([ingredients['short'][i].split() for i in range(ingredients.shape[0])])


### Fill the non-train short values based either on word, or the POS probability

In [47]:
samples=[]
idx=ingredients[ingredients.train==1].index

In [48]:
for i in idx:
    split=ingredients.loc[i]['split']
    pos=ingredients.loc[i]['pos']
    short_split=ingredients.loc[i]['short_split']
    for j in range(len(split)):
        samples.append([split[j], pos[j], int(split[j] in short_split)])

In [49]:
samples=pd.DataFrame(samples, columns=['word', 'pos','keep'])

In [50]:
samples.shape

(1538, 3)

In [51]:
samples.head()

,word,pos,keep
0,cremini,NN,1
1,mushrooms,NNS,1
2,sliced,VBN,0
3,mushrooms,NNS,0
4,rice,NN,1


In [ ]:
### Simple probabiity model

def keep_prob(x):
    if x[0] in list(samples.word):
        idx=samples[samples.word==x[0]].index
        if x[1] in list(samples.loc[idx]['pos']):
            idx=samples.loc[idx][samples.pos==x[1]].index
        else:
            idx=samples[samples.pos==x[1]].index
    else: 
        idx=samples[samples.pos==x[1]].index
    
    try: return sum(samples.loc[idx]['keep'])/len(samples.loc[idx]['keep']) 
    except: return 0

In [ ]:
# Fille the 'short_spit' for non-train data

ingredients['short_split']=pd.Series([ingredients['short'][i].split() for i in range(ingredients.shape[0])])

for i in ingredients[ingredients.train==0].index:
    ss=[]
    for j in range(len(ingredients.loc[i]['split'])):
        x=[ingredients.loc[i]['split'][j], ingredients.loc[i]['pos'][j]]
        if keep_prob(x)>0.5:
            ss.extend([x[0]])
    ingredients.loc[i, 'short_split'].extend(ss)


In [ ]:
# Add 'short' name and 'stemmed' name columns from short_split

ingredients['short']=pd.Series([' '.join(x[1:]) if x[0]=='na' else ' '.join(x) 
                               for x in ingredients['short_split'] ])

def stem(x):
    try: return lemmatizer.lemmatize(x)
    except: return None
    

ingredients['stem']=pd.Series([stem(x) for x in ingredients['short']])

In [53]:
ingredients.head()

,full name,stem,short,split,pos,train,short_split
0,cremini mushrooms,cremini mushrooms,cremini mushrooms,"[cremini, mushrooms]","[NN, NNS]",1,"[cremini, mushrooms]"
1,caramel,caramel,caramel,[caramel],[NN],0,"[na, caramel]"
2,maldon sea salt,maldon sea salt,maldon sea salt,"[maldon, sea, salt]","[NNS, NN, NN]",0,"[na, maldon, sea, salt]"
3,orecchiette,orecchiette,orecchiette,[orecchiette],[NN],0,"[na, orecchiette]"
4,baby bok choy,baby bok choy,baby bok choy,"[baby, bok, choy]","[NN, NNS, VBP]",0,"[na, baby, bok, choy]"


In [ ]:
ingredients.to_csv('ingredients.csv')

In [141]:
short_ingredients=list(set(ingredients['stem']))
short_ingredients=[x for x in short_ingredients if type(x)==str]

In [142]:
len(short_ingredients)

2640

In [143]:
short_ingredients.remove('meal')
short_ingredients.remove('fat')

In [144]:
pd.DataFrame(short_ingredients, columns=['ingredient']).to_csv("data/ingr_list.csv", encoding="UTF-8")

In [ ]:
# Further upload of ingredients dataset

In [114]:
ingredients=pd.read_csv('data/ingredients.csv')
ingredients.drop('Column1', axis=1, inplace=True)

In [222]:
from ast import literal_eval

In [115]:
for column in list(['split', 'pos', 'short_split']):
    ingredients[column]=pd.Series(literal_eval(x) for x in ingredients[column])


### Create the dataset out of 2.7K unified ingredients and recipes

In [312]:
data.rename(columns={'fat': 'fats'}, inplace=True)

In [314]:
# read the full list of 3700 ingredients
ingredients=pd.read_csv('data/ingredients.csv')
ingredients.drop('Column1', axis=1, inplace=True)

In [310]:
ingredients.columns=['full', 'stem', 'subs1', 'subs2']

In [315]:
ingredients.head()

,full name,stem,substitute 1,substitute 2
0,cremini mushrooms,crimini mushrooms,common mushroom,NaN
1,caramel,caramel,NaN,NaN
2,maldon sea salt,sea salt,salt,NaN
3,orecchiette,orecchiette,NaN,NaN
4,baby bok choy,bok choy,NaN,NaN


In [289]:
data=pd.read_csv('data/epi_data.csv')

In [ ]:
data.rename(columns={'fat': 'fats'}, inplace=True)

In [316]:
data_short=data[['title','calories','protein','carbs','fats','sodium','steps']]

In [317]:
short_ingredients=list(set(ingredients['stem']))

In [318]:
short_ingredients[:5]

['maitake mushrooms', 'america', 'shiro miso', 'hash brown', 'jicama']

In [321]:
for ingr in short_ingredients:
    items=[ingredients.loc[i]['full name'] for i in ingredients.index if ingredients.loc[i]['stem']==ingr]
    items=[x for x in items if x in data.columns]
    data_short[ingr]=data[items].sum(axis=1)


C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [322]:
data_short.shape

(20111, 2595)

In [323]:
data_short[short_ingredients]=data_short[short_ingredients].applymap(lambda x: int(x>0))

C:\Users\vnuretdi\Anaconda3\lib\site-packages\pandas\core\frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [ ]:
data_short.drop('other', axis=1, inplace=True)
data_short.drop('no ingredient', axis=1, inplace=True)

In [324]:
data_short.to_csv('data_short.csv', encoding="UTF-8")

### Categorize ingredients

#### Categorize vegetarian and vegan products

In [268]:
data_short=pd.read_csv('data/data_short.csv', encoding="UTF-8")

In [272]:
data_short.drop('Unnamed: 0', axis=1, inplace=True )

In [269]:
short_ingredients=pd.read_csv('data/ingr_list.csv', encoding="UTF-8")

In [270]:
short_ingredients.drop('Unnamed: 0', axis=1, inplace=True)

In [325]:
red_meat=['lamb', 'beef', 'meat', 'mutton', 'veal', 'venison', 'pork',
      'turkey', 'bacon', 'ham', 'hot dogs', 'jamon', 'prosciutto', 'salami', 'sausages', 'rabbit']

In [ ]:
short_ingredients[str(column)]=pd.Series([1 if len(set(column).intersection(x.split(' ')))>0 else 0 
                                     for x in short_ingredients.ingredient])

In [152]:
poultry=['chicken', 'turkey', 'duck', 'grouse', 'pheasant']

In [153]:
fish=['fish', 'salmon', 'cod', 'tilapia', 'catfish', 'halibut', 'mahi', 'tuna', 'mackerel',  
      'swordfish', 'sole', 'bass', 'anchovy', 'herring', 'haddock', 'monkfish', 'mullet', 'sardines', 'swordfish', 
     'trout', 'caviar', 'turbot', 'snapper', 'sturgeon', 'flounder', 'barramundi']

In [154]:
seafood=['crabs', 'shells', 'shrimp', 'clam', 'lobster', 'octopus', 'scallops', 'squid', 'eel']

In [155]:
dairy=['cheese', 'milk', 'mascarpone', 'butter' 'buttermilk', 'ice cream', 'sour cream', 'yogurt', 'yoghurt', 'kefir', 'custard']

In [156]:
milk_exclude=['coconut', 'soymilk', 'soy', 'cashew', 'almond', 'rice']

In [157]:
egg=['egg', 'eggs', 'yolk', 'whites']

In [160]:
#vegetables
vegetables=list(pd.read_csv('data/vegetables.csv')['Vegetables'])
vegetables=[x.lower() for x in vegetables]
vegetables=[x.split('/') for x in vegetables]
vegetables=[y for x in vegetables for y in x]

In [197]:
# fruit
fruits=list(pd.read_csv('data/fruits.csv')['Fruits'])
fruits=[x.lower() for x in fruits]
fruits=[x.split('/') for x in fruits]
fruits=[y for x in fruits for y in x]

In [ ]:
d=dict(zip(['red_meat', 'poultry', 'fish', 'seafood', 'egg', 'vegetables', 'fruits'], 
     [red_meat, poultry, fish, seafood, egg, vegetables, fruits]))

In [229]:
for column in d.keys():
    short_ingredients[column]=pd.Series([1 if len(set(d[column]).intersection(x.split(' ')))>0 else 0 
                                     for x in short_ingredients.ingredient])

In [233]:
short_ingredients['dairy']=pd.Series([1 if (len(set(dairy).intersection(x.split(' ')))>0)&
                                            (len(set(milk_exclude).intersection(x.split(' ')))==0) else 0 
                                     for x in short_ingredients.ingredient])

In [234]:
short_ingredients.head()

,ingredient,red_meat,poultry,fish,seafood,egg,vegetables,fruits,dairy
0,maitake mushrooms,0,0,0,0,0,1,0,0
1,america,0,0,0,0,0,0,0,0
2,shiro miso,0,0,0,0,0,0,0,0
3,hash brown,0,0,0,0,0,0,0,0
4,jicama,0,0,0,0,0,0,0,0


In [241]:
short_ingredients.to_csv('data/ingr_list.csv', encoding='UTF-8')

In [239]:
[(x, sum(short_ingredients[x])) for x in ['red_meat', 'poultry', 'fish', 'seafood', 'egg', 'vegetables', 'fruits', 'dairy']]

[('red_meat', 137),
 ('poultry', 46),
 ('fish', 47),
 ('seafood', 33),
 ('egg', 12),
 ('vegetables', 320),
 ('fruits', 227),
 ('dairy', 67)]

In [273]:
data_short.head()

,title,calories,protein,carbs,fat,sodium,steps,meal,shredded parmesan cheese,tea,...,cointreau,pickled radish,liquid,red bliss potato,garlic chives,shelled pistachio,chive blossoms,baking apples,haddock,chili
0,"Lentil, Apple, and Turkey Wrap",426.0,30.0,60.75,7.0,559.0,3,lunch/dinner,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Boudin Blanc Terrine with Red Onion Confit,403.0,18.0,31.00,23.0,1439.0,5,lunch/dinner,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Potato and Fennel Soup Hodge,165.0,6.0,19.50,7.0,165.0,2,lunch/dinner,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mahi-Mahi in Tomato Olive Sauce,NaN,NaN,NaN,NaN,NaN,2,dinner,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Spinach Noodle Casserole,547.0,20.0,44.75,32.0,452.0,1,lunch/dinner,0,0,...,0,0,0,0,0,0,0,0,0,0


In [280]:
freq=[(x, sum(data_short[x])) for x in 
      data_short.drop(['title','calories','protein','carbs','fat','sodium','steps', 'meal'], axis=1).columns]

In [281]:
freq.sort(key=lambda tup: tup[1])

In [284]:
freq[::-1][:100]

[('olive oil', 6587),
 ('garlic', 5825),
 ('butter', 5741),
 ('salt', 5195),
 ('pepper', 5105),
 ('sugar', 5004),
 ('onion', 4644),
 ('egg', 4030),
 ('water', 3545),
 ('flour', 3418),
 ('lemon juice', 3066),
 ('oil', 2348),
 ('kosher salt', 1773),
 ('chicken broth', 1753),
 ('milk', 1683),
 ('shallot', 1512),
 ('cinnamon', 1404),
 ('dry wine', 1402),
 ('lime juice', 1389),
 ('wine vinegar', 1365),
 ('chicken', 1355),
 ('brown sugar', 1343),
 ('parsley', 1324),
 ('carrot', 1301),
 ('tomato', 1287),
 ('whipping cream', 1206),
 ('no ingredient', 1178),
 ('cream', 1173),
 ('cilantro', 1144),
 ('vanilla extract', 1112),
 ('ginger', 1032),
 ('parmesan', 973),
 ('dijon mustard', 930),
 ('baking powder', 914),
 ('thyme', 910),
 ('soy sauce', 905),
 ('red bell pepper', 899),
 ('honey', 891),
 ('cumin', 879),
 ('celery', 875),
 ('lemon', 867),
 ('bread', 839),
 ('scallion', 767),
 ('peeled ginger', 746),
 ('vinegar', 728),
 ('sour cream', 726),
 ('orange juice', 718),
 ('mayonnaise', 710),
 ('ch